In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv('.\\data\\digit-recognizer\\train.csv')
split_ratio = 0.8
total_size = len(data)
train_size = int(split_ratio * total_size)
train_data = data[:train_size]
test_data = data[train_size:]
print(len(train_data))
print(len(test_data))

33600
8400


In [3]:
y_train = train_data['label']
x_train = train_data.drop(['label'], axis = 1)

In [4]:
def get_batched_data():
    current_pos = 0
     
    total_size = len(x_train)
    steps_per_epoch = total_size / batch_size
    
    def one_hot_encode_y(y_value):
        ohe_vector = np.zeros((10, ))
        ohe_vector[y_value] = 1
        return ohe_vector
    
    for i in np.arange(steps_per_epoch):
        x_batched = x_train[current_pos: current_pos + batch_size].values
        x_batched_reshaped = np.array([x.reshape((28, 28, 1)) for x  in x_batched])#list(map(lambda x: x.reshape((28, 28)), x_batched)))
        y_batched = y_train[current_pos: current_pos + batch_size].values.tolist()
        y_batched_encoded = np.array([one_hot_encode_y(y) for y in y_batched])#list(map(lambda x: one_hot_encode_y(x), y_batched)))
        current_pos += batch_size
        yield x_batched_reshaped, y_batched_encoded

In [5]:
batch_size = 100
epochs = 3

In [6]:
input_image = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 1], name = 'input_image')
y_actual = tf.placeholder(dtype = tf.float32, shape = [None, 10], name = 'y_actual')

conv1_w = tf.get_variable('conv1_w', shape = [3, 3, 1, 32], initializer = tf.contrib.layers.xavier_initializer())
conv1_b = tf.get_variable('conv1_b', initializer = tf.constant(0.1, shape = [32]))

conv1 = tf.nn.conv2d(input_image, conv1_w, strides = [1, 1, 1, 1], padding = 'VALID') + conv1_b
out1 = tf.nn.relu(conv1, name = 'relu1')

maxpool1 = tf.nn.max_pool(out1, ksize = [1, 2, 2, 1], strides= [1, 1, 1, 1], padding = 'VALID', name = 'maxpool1')

conv2_w = tf.get_variable('conv2_w', shape = [3, 3, 32, 64], initializer = tf.contrib.layers.xavier_initializer())
conv2_b = tf.get_variable('conv2_b', initializer = tf.constant(0.1, shape = [64]))

conv2 = tf.nn.conv2d(maxpool1, conv2_w, strides = [1, 1, 1, 1], padding = 'VALID') + conv2_b
out2 = tf.nn.relu(conv2, name = 'relu2')

maxpool2 = tf.nn.max_pool(out2, ksize = [1, 2, 2, 1], strides = [1, 1, 1, 1], padding = 'VALID', name = 'maxpool2')

norm_out2 = tf.contrib.layers.batch_norm(maxpool2)

conv3_w = tf.get_variable('conv3_w', shape = [3, 3, 64, 128], initializer = tf.contrib.layers.xavier_initializer())
conv3_b = tf.get_variable('conv3_b', initializer = tf.constant(0.1, shape = [128]))

conv3 = tf.nn.conv2d(norm_out2, conv3_w, strides = [1, 1, 1, 1], padding = 'VALID') + conv3_b
out3 = tf.nn.relu(conv3, name = 'relu3')

out3_flattened = tf.reshape(out3, [-1, 51200])

fc4_w = tf.get_variable('fc4_w', shape = [51200, 10], initializer = tf.contrib.layers.xavier_initializer())
fc4_b = tf.get_variable('fc4_b', initializer = tf.constant(0.1, shape = [10]))

y_pred = tf.nn.softmax(tf.matmul(out3_flattened, fc4_w) + fc4_b, name = 'y_pred')

loss = tf.reduce_mean(-tf.reduce_sum(y_actual * tf.log(y_pred), [1]))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
sess = tf.InteractiveSession()

optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
for epoch in np.arange(epochs):
    step_count = 0
    for x_batched, y_batched in get_batched_data():
        sess.run(train_step, feed_dict = {input_image: x_batched, y_actual: y_batched})
        
        if step_count % 10 == 0:
            print('Running iteration for epoch: {}, step_count: {}'.format(epoch, step_count))
            
        step_count += 1    

Running iteration for epoch: 0, step_count: 0
Running iteration for epoch: 0, step_count: 10
Running iteration for epoch: 0, step_count: 20
Running iteration for epoch: 0, step_count: 30
Running iteration for epoch: 0, step_count: 40
Running iteration for epoch: 0, step_count: 50
Running iteration for epoch: 0, step_count: 60
Running iteration for epoch: 0, step_count: 70
Running iteration for epoch: 0, step_count: 80
Running iteration for epoch: 0, step_count: 90
Running iteration for epoch: 0, step_count: 100
Running iteration for epoch: 0, step_count: 110
Running iteration for epoch: 0, step_count: 120
Running iteration for epoch: 0, step_count: 130
Running iteration for epoch: 0, step_count: 140
Running iteration for epoch: 0, step_count: 150
Running iteration for epoch: 0, step_count: 160
Running iteration for epoch: 0, step_count: 170
Running iteration for epoch: 0, step_count: 180
Running iteration for epoch: 0, step_count: 190
Running iteration for epoch: 0, step_count: 200
Run

In [9]:
y_test = test_data['label']
x_test = test_data.drop(['label'], axis = 1)

In [10]:
def get_test_data():
    
    x_batched = x_test.values
    x_batched_reshaped = np.array([x.reshape((28, 28, 1)) for x  in x_batched])
    y_batched = y_test.values
    return x_batched_reshaped, y_batched

In [11]:
test_x_mod, test_y_mod = get_test_data()

In [13]:
predictions = []
for x in test_x_mod:
    result = sess.run(y_pred, feed_dict={input_image: [x]})
    predictions.append(result)

In [14]:
print(predictions[0])

[[9.9992549e-01 6.7154473e-09 1.0675867e-06 7.7174184e-11 1.2788688e-10
  2.1589262e-08 7.3368363e-05 1.0281833e-08 1.4358060e-08 2.6773835e-08]]


In [15]:
test_y_predictions = [x.argmax() for x in predictions]

In [17]:
print(test_y_predictions[0:10])

[0, 7, 7, 2, 2, 6, 5, 7, 8, 5]


In [18]:
correct_predictions = 0
total_count = len(test_y_mod)

for i in np.arange(total_count):
    if test_y_predictions[i] == test_y_mod[i]:
        correct_predictions = correct_predictions + 1

accuracy = correct_predictions / total_count
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.9383333333333334
